In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np


#hyper parameters
batch_size = 50
learning_rate = 0.001
num_class = 10
num_epoch = 10

# cifar10 dataset
train_dataset = dsets.CIFAR10(root='./data', 
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = dsets.CIFAR10(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
# each block's channel 
cfg = {
    'cfg_0' : [64, 64, 256],
    'cfg_1' : [128, 128, 512],
    'cfg_2' : [256, 256, 1024],
    'cfg_3' : [512, 512, 2048]
}

In [3]:
class bottleNeck(nn.Module):
    expansion = 4
    
    def __init__(self, in_channel, cfg, stride=1):
        super(bottleNeck, self).__init__()
        self.unit = nn.Sequential(
                        nn.Conv2d(in_channel, cfg[0], kernel_size=1, bias=False),
                        nn.BatchNorm2d(cfg[0]),   
                        nn.ReLU(inplace=True),
                        nn.Conv2d(cfg[0], cfg[1], kernel_size=3, padding=1, stride=stride, bias=False),
                        nn.BatchNorm2d(cfg[1]), 
                        nn.ReLU(inplace=True),
                        nn.Conv2d(cfg[1], cfg[2], kernel_size=1, bias=False),
                        nn.BatchNorm2d(cfg[2]) 
                    )
        self.stride = stride
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channel != self.expansion * cfg[0]:
            self.shortcut = nn.Sequential(
                                nn.Conv2d(in_channel, self.expansion * cfg[0], kernel_size=1, stride=stride, bias=False),
                                nn.BatchNorm2d(self.expansion * cfg[0])
                            )
        
    def forward(self, x):
        out = self.unit(x)
        residual = self.shortcut(x)
        out += residual
        out = F.relu(out)
        return out

In [4]:
class resNet152(nn.Module):
    def __init__(self, block, cfg, num_block):
        super(resNet152, self).__init__()
        self.in_ch = 64
        self.go = nn.Sequential(
                    nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1, bias=False),
                    nn.BatchNorm2d(64),
                    nn.ReLU(inplace=True)
                )
        self.cfg_0 = self.make_residual_block(block, cfg['cfg_0'], num_block[0], stride=1)
        self.cfg_1 = self.make_residual_block(block, cfg['cfg_1'], num_block[1], stride=2)
        self.cfg_2 = self.make_residual_block(block, cfg['cfg_2'], num_block[2], stride=2)
        self.cfg_3 = self.make_residual_block(block, cfg['cfg_3'], num_block[3], stride=2)
        self.fc = nn.Linear(1 * 1 * 2048, 1000)

    def forward(self, image):
        out = self.go(image)
        out = self.cfg_0(out)
        out = self.cfg_1(out)
        out = self.cfg_2(out)
        out = self.cfg_3(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
    def make_residual_block(self, block, cfg, num_block, stride):
        strides = [stride] + [1]*(num_block-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_ch, cfg, stride))
            self.in_ch = cfg[0] * block.expansion
        return nn.Sequential(*layers)
    

In [5]:
MODEL = resNet152(bottleNeck, cfg, num_block=[3, 8, 36, 3])
MODEL.cuda(1)
print (MODEL)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MODEL.parameters(), lr=learning_rate)

resNet152(
  (go): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
  )
  (cfg_0): Sequential(
    (0): bottleNeck(
      (unit): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU(inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (5): ReLU(inplace)
        (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): bottleNeck(
      (u

In [6]:
# training session
batch_count = 0

for epoch in range(num_epoch):
    for i, (image, label) in enumerate(train_loader):
        images = Variable(image.cuda(1))
        labels = Variable(label.cuda(1))
        
        optimizer.zero_grad()
        out = MODEL(images)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        
        if(i+1) % 100 == 0:
            batch_count += 1
            print('training... %d%%, Loss: %.4f' % ( ((batch_size * 100) * batch_count * 100) / (num_epoch * len(train_dataset)), loss.data[0]))

training... 1%, Loss: 2.0195
training... 2%, Loss: 1.9498
training... 3%, Loss: 1.5976
training... 4%, Loss: 1.6562
training... 5%, Loss: 1.7822
training... 6%, Loss: 1.5308
training... 7%, Loss: 1.3791
training... 8%, Loss: 1.2847
training... 9%, Loss: 1.4335
training... 10%, Loss: 1.2289
training... 11%, Loss: 1.2006
training... 12%, Loss: 1.0834
training... 13%, Loss: 1.0114
training... 14%, Loss: 1.0561
training... 15%, Loss: 1.1488
training... 16%, Loss: 1.3212
training... 17%, Loss: 1.0230
training... 18%, Loss: 0.9633
training... 19%, Loss: 1.2706
training... 20%, Loss: 1.0237
training... 21%, Loss: 0.8851
training... 22%, Loss: 0.9199
training... 23%, Loss: 0.8458
training... 24%, Loss: 1.2557
training... 25%, Loss: 0.8571
training... 26%, Loss: 0.6821
training... 27%, Loss: 0.6420
training... 28%, Loss: 0.9060
training... 29%, Loss: 0.8498
training... 30%, Loss: 0.6302
training... 31%, Loss: 0.7148
training... 32%, Loss: 1.1695
training... 33%, Loss: 0.6594
training... 34%, Lo

In [7]:
total = 0
correct = 0

for (image, label) in test_loader:
    images = Variable(image.cuda(1))
    
    output = MODEL(images)
    _, predicted = torch.max(output.data, 1)
    correct += (predicted.cpu()==label).sum()
    total += labels.size(0)

print('Accuracy : %d%%' %((100*correct)/total))

Accuracy : 95%
